# 🚀 Train Your First Custom Wake Word with Nanowakeword!

Welcome to the official tutorial for **Nanowakeword**! 

In this notebook, we will guide you through the entire process of training a high-performance, custom wake word model from scratch. You don't need any pre-existing data—we will download everything we need and let Nanowakeword's intelligent engine do the heavy lifting.

**Our goal:** Go from zero to a ready-to-use wake word model in just a few simple steps. Let's get started!

**Installation**

In [ ]:
# @title Step 1: Install Nanowakeword
# We install the full [train] package to get all the necessary dependencies.

! pip install --no-cache-dir "nanowakeword[train]==1.2.0"
! pip install piper-tts

print("Installation complete!")

## Step 2: Prepare the Dataset

A great model starts with great data. For this tutorial, we will:
1.  **Download** open-source noise and Room Impulse Response (RIR) datasets.
2.  **Generate** our own custom wake word samples using a built-in TTS engine.
3.  **Organize** all your project files within a clean, well-structured folder hierarchy for better clarity and maintainability.

In [3]:
# @title Step 2.1: Download & Prepare the Nanowakeword Starter Dataset

import os
from pathlib import Path
import subprocess
import shutil

# --- Configuration ---
DATASET_REPO_URL = "https://huggingface.co/datasets/arcosoph/SonicWeave-v1"
DATA_DIR = Path("./nanowakeword_data")

# --- Define Final Paths ---
noise_dir = DATA_DIR / "Noise"
rir_dir = DATA_DIR / "Rir"
positive_dir = DATA_DIR / "positive_wakeword"
negative_dir = DATA_DIR / "negative_speech"

# --- Main Logic ---
print("The dataset is downloading. This may take a moment...")

# Download only if the dataset folders are not already created.
if not noise_dir.exists() or not rir_dir.exists() or not any(noise_dir.iterdir()):
    
    # Clone the repository to a temporary location
    temp_clone_dir = DATA_DIR / "temp_repo"
    
    print(f"Downloading the Starter Dataset from {DATASET_REPO_URL}...")
    
    # --depth 1 only downloads the latest commit, which is much faster      
    try:
        subprocess.run(
            ["git", "clone", "--depth", "1", DATASET_REPO_URL, str(temp_clone_dir)],
            check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL # Hide unnecessary log messages
        )
    except subprocess.CalledProcessError as e:
        print(f"Error: Failed to clone the dataset repository. Please check the URL.")
        print(f"Git command failed with error: {e}")
    else:
        print("Organizing dataset files...")

        # Move the noise and rir folders from the cloned repository to the correct location
        try:

            # Move only required folders
            for folder_name in ["Noise", "Rir"]:
                src = temp_clone_dir / folder_name
                dst = DATA_DIR / folder_name
                if src.exists():
                    shutil.move(str(src), str(dst))

            # Delete temp_repo, ignore errors if some files are locked
            shutil.rmtree(temp_clone_dir, ignore_errors=True)
                        
            print("\nStarter Dataset is ready!")
        except FileNotFoundError:
            print("Error: 'noise' or 'rir' folder not found inside the cloned repository.")
        except Exception as e:
            print(f"Error organizing files: {e}")
else:
    print("Nanowakeword Starter Dataset already found.")

The dataset is downloading. This may take a moment...
Nanowakeword Starter Dataset already found.


**Generate Wake Word Samples**

In [ ]:
# @title Step 2.2: Generate Custom Wake Word & Adversarial Negative Audio

from nanowakeword.generate_samples import generate_samples
from nanowakeword.data import generate_adversarial_texts 

#@markdown Define your custom wake word and the number of samples you want to generate.
WAKE_WORD = "Hey Computer"    #@param {type:"string" }
NUM_POSITIVE_SAMPLES = 1500   #@param {type:"integer"}
NUM_NEGATIVE_SAMPLES = 4000   #@param {type:"integer"}
#    ✍️(◔◡◔) ༼ つ ◕_◕ ༽つ


# 1. Creating positive samples (directly)
generate_samples(
                text=WAKE_WORD,
                output_dir=str(positive_dir),
                max_samples=NUM_POSITIVE_SAMPLES
          )

print(f"\nGenerating {NUM_NEGATIVE_SAMPLES} intelligent adversarial negative samples...")


# NanoWakeword will automatically generate strong negative text based on the wakeword.
# For example: "Hey Commuter", "Play Computer", "Hey Peter", "Okay Jupiter" etc. Thousands of variations
adversarial_texts = generate_adversarial_texts(
                    input_text=WAKE_WORD,
                    N=NUM_NEGATIVE_SAMPLES
)

# Now create audio from those automatically generated texts
generate_samples(
                 text=adversarial_texts,
                 output_dir=str(negative_dir),
                 max_samples=NUM_NEGATIVE_SAMPLES
)

print("\nAll synthetic audio has been generated successfully!")

e:\t\.venv\Lib\site-packages\pronouncing\__init__.py:3: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream
e:\t\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\t\.venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902

Generating Audio: 100%|██████████| 15/15 [00:03<00:00,  4.21it/s]



Generating 40 intelligent adversarial negative samples...


Generating Audio: 100%|██████████| 40/40 [00:04<00:00,  8.52it/s]


All synthetic audio has been generated successfully!


## Step 3: Configure and Train the Model

Now for the fun part! We will create a `config.yaml` file and then run the Nanowakeword training command.

We will use the magical `--auto-config` flag to let the Intelligent Engine analyze our newly prepared data and build the best possible model.

**Configuration and Training**

In [4]:
# @title Step 3.1: Create the Configuration File
import yaml

config_dict = {
    # Data Paths (pointing to our newly created folders)
    "wakeword_data_path": str(positive_dir),
    "background_data_path": str(negative_dir),
    "background_paths": [str(noise_dir)],
    "rir_paths": [str(rir_dir)],
    # Model Output
    "model_name": "hey_computer_v1",
    "output_dir": "./trained_models",
    # Model Type
    "model_type": "dnn" # A good default for many tasks
}

# Write the config to a YAML file
config_path = "./config.yaml"
with open(config_path, 'w') as f:
    yaml.dump(config_dict, f, default_flow_style=False)

print(f"✅ Configuration file saved to {config_path}")

✅ Configuration file saved to ./config.yaml


**Run Training!**

In [6]:
# # @title Step 3.2: Run the Magic Command! 🚀
# # This command will do everything: augment data, extract features, and train the model.
# # It might take some time depending on the hardware (especially on a CPU).

# # !nanowakeword-train --training_config {config_path} --auto-config --augment_clips  --train_model

from nanowakeword.trainer import train 

args_list = [
    '--training_config', f'{config_path}',
    '--auto-config',
    '--augment_clips',
    '--train_model',
    '--overwrite' 
]

print("Starting NanoWakeWord training...")

try:
    train(args_list)
    print("\n\nCONGRATULATIONS! (✿◕‿◕✿)")
    print("Your custom wake word model has been successfully trained!")

except Exception as e:
    print(f"\nAn error occurred during training: {e}")

e:\t\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
e:\t\.venv\Lib\site-packages\speechbrain\utils\torch_audio_backend.py:57: UserWarning: torchaudio._backend.list_audio_backends has been deprecated. This deprecation is part of a large refactoring effort to transition TorchAudio into a maintenance phase. The decoding and encoding capabilities of PyTorch for both audio and video are being consolidated into TorchCodec. Please see https://github.com/pytorch/audio/issues/3902 for more information. It will be removed from the 2.9 release. 
  available_backends = torchaudio.list_audio_backends()
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Regis

Starting NanoWakeWord training...


  _   _               __          __   _     __          __           _ 
 | \ | |              \ \        / /  | |    \ \        / /          | |
 |  \| | __ _ _ __   __\ \  /\  / /_ _| | ____\ \  /\  / /__  _ __ __| |
 | . ` |/ _` | '_ \ / _ \ \/  \/ / _` | |/ / _ \ \/  \/ / _ \| '__/ _` |
 | |\  | (_| | | | | (_) \  /\  / (_| |   <  __/\  /\  / (_) | | | (_| |
 |_| \_|\__,_|_| |_|\___/ \/  \/ \__,_|_|\_\___| \/  \/ \___/|_|  \__,_|

----------------------------------------

STEP 1: Verifying and Preprocessing Data Directories

====================================================

INFO:root:Verifying and preprocessing audio files in: nanowakeword_data\negative_speech
Processing negative_speech: 100%|██████████| 40/40 [00:00<00:00, 57.89it/s]
INFO:root:Finished processing directory: nanowakeword_data\negative_speech. Converted 0 files.
INFO:root:Verifying and preprocessing audio files in: nanowakeword_data\Noise
Processing Noise:  76%|███████▌  | 1037/1365 [00:21<00:06, 48.75it/s]


KeyboardInterrupt: 

## Step 4: What's Next?

You have successfully trained your own custom wake word model!

You can now download the `.onnx` or `.tflite` file from the `trained_models` directory (check the file browser on the left) and use it in your own applications.

For more advanced topics, such as using your own datasets or fine-tuning the configuration, please check out our full documentation on **[GitHub](https://github.com/arcosoph/nanowakeword)**.